Import libraries and define main constants

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

TOP_UNIVERSITIES_BASE_URL = "https://www.topuniversities.com"
TIMES_BASE_URL = "https://www.timeshighereducation.com/"
# Max rank to load for each ranking
MAX_RANK=200 # 200

## Cached requests

The following method adds a cache for simple HTTP GET requests and ensures that at most one request will be sent per unique URL.

**Re-evalute the cell with the function to flush the cache.**

In [2]:
CACHE = {}
def cached_get_request(url, cache=CACHE):
    """
    Perform a cached HTTP GET request for `url`.
    
    The first time you send a request for an `url`, it will effectivelly
    send an HTTP request. The response will be cached in `cache` and each
    subsequent call for a cached URL will use a cache lookup using an exact
    match on the url instead of sending a new HTTP request.
    
    This function will use a shared cache by default, you can provide your
    own dictionary if you want to override this behavior.
    Reevalute this function to flush the shared cache.
    
    :param url str: URL to GET
    :cache: A dictionary from urls to HTTP responses. Mutated if it did not contain the key `url`.
    :return: _requests_' response for the provided URL.
    """
    
    if url not in cache:
        print("send request : "+url)
        cache[url] = requests.get(url)
    return cache[url]

We define a few functions used to read the data
- `read_rank` allows to convert a string of the form `"=40"` to `40`, `"10-12"` to `10` (TODO: give example with em dash)
- `read_us_formatted_integer` allows to convert string representation with a comma used used as the thousands separtor.

In [12]:
def read_rank(raw_rank):
    """
    :param raw_rank str: Rank string, as found by scrapping
    :return int: Integer representing the rank of the university
    """
    if '-' in raw_rank:
        return int(raw_rank[:raw_rank.index('-')])
    if '–' in raw_rank:
        return int(raw_rank[:raw_rank.index('–')])
    return int(raw_rank.replace('=', ''))

In [13]:
def read_us_formatted_integer(raw_integer):
    """
    Reads a string representing an integer properly formatted using US convention.
    
    The US format uses commas as the thousands separator.
    
    Examples:
    ```
    >>> read_us_formatted_integer("0")
    0
    >>> read_us_formatted_integer("1,000")
    1000
    >>> read_us_formatted_integer("999,999,999")
    999999999
    
    ```
    :param raw_integer str: US Formatted string for an integer
    :return int: Integer corresponding to the provided value.
    """
    return int(raw_integer.replace(",", ""))

In [14]:
def read_percentage(raw_percentage):
    """
    :param raw_percentage str: Percentage string
    :return float: Ratio corresponding to the percentage (between 0 and 1)
    """ 
    return float(raw_percentage.replace("%", "")) / 100

TODO : Describe data

In [15]:
def make_entry(name, rank, country, region, faculty_total, faculty_international, students_total, students_international, **extra_keys):
    return {
        "name": name,
        "rank": rank,
        "country": country,
        "region": region,
        "faculty_total": faculty_total,
        "faculty_international": faculty_international,
        "students_total": students_total,
        "students_international": students_international,
        **extra_keys,
    }

In [16]:
def get_top(rawData, limit, top_uni):
    # TODO: Replace this function if we need to be flexible
    # TODO: Define "flexible"
    # TODO: Convert this function to a generator and let the consumer decide if he wants to allocate an array
    top = []
    for r in rawData:
#         print r
        tempRank = read_rank(r["rank_display"] if top_uni else r["rank"])
        if tempRank > limit:
            break
        top.append(r)
    return top

Fetch data from Top Universities
TODO : try catch beautiful soup
Make function for scrapping

In [25]:
def findValueHtml(custom_class, html_to_parse):
    try:
        value_inside = html_to_parse.find("div", {"class": custom_class}).find("div", {"class": "number"}).text.strip()
    except Exception as e:
        #print("Error parsing html : ")
        #print(html_to_parse)
        #print(e)
        return None
    return value_inside

In [35]:
ranking_response = cached_get_request("https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051.txt")
raw_ranking_data = ranking_response.json()['data']

def read_top_universities_entry(raw_entry, html_details):
    name = raw_entry["title"]
    rank = read_rank(raw_entry["rank_display"])
    country = raw_entry["country"]
    region = raw_entry["region"]
    faculty_total_str = findValueHtml("total faculty",html_details)
    faculty_total = read_us_formatted_integer(faculty_total_str) if faculty_total_str is not None else None
    faculty_international_str = findValueHtml("inter faculty",html_details)
    faculty_international = read_us_formatted_integer(faculty_international_str) if faculty_international_str is not None else None
    students_total_str  = findValueHtml("total student",html_details)
    students_total = read_us_formatted_integer(students_total_str) if students_total_str is not None else None
    students_international_str  = findValueHtml("total inter",html_details)
    students_international = read_us_formatted_integer(students_international_str) if students_international_str is not None else None
    
    return make_entry(name, rank, country, region, faculty_total, faculty_international, students_total, students_international)


def get_top_universities_data(raw_ranking_data, max_rank):
    """
    Return the list of parsed enries (with details) of the top universities ranking.
    It may perform network requests.
    """
    for raw_entry in raw_ranking_data:
        entry = {}
        
        rank = read_rank(raw_entry["rank_display"])
        if (rank > max_rank):
            # Assume sorted raw ranks to break (instead of `continue`-ing)
            break
        
        details_reponse = cached_get_request(TOP_UNIVERSITIES_BASE_URL + raw_entry["url"])
        raw_details_data = BeautifulSoup(details_reponse.text, 'html.parser')

        yield read_top_universities_entry(raw_entry, raw_details_data)


def get_top_universities_df(raw_ranking_data, max_rank):
    return pd.DataFrame([*get_top_universities_data(raw_ranking_data, max_rank)])

top_universities_df = get_top_universities_df(raw_ranking_data, MAX_RANK)
top_universities_df[40:80]

,country,faculty_international,faculty_total,name,rank,region,students_international,students_total
40,South Korea,147.0,1250.0,KAIST - Korea Advanced Institute of Science & ...,41,Asia,584.0,9826.0
41,Australia,1477.0,3311.0,The University of Melbourne,41,Oceania,18030.0,42182.0
42,France,75.0,178.0,"Ecole normale supérieure, Paris",43,Europe,374.0,1907.0
43,United Kingdom,942.0,2870.0,University of Bristol,44,Europe,5099.0,20630.0
44,Australia,1612.0,2924.0,The University of New South Wales (UNSW Sydney),45,Oceania,14292.0,39784.0
45,Hong Kong,1074.0,2208.0,The Chinese University of Hong Kong (CUHK),46,Asia,4824.0,18037.0
46,Australia,1870.0,3158.0,The University of Queensland,47,Oceania,10420.0,37497.0
47,United States,425.0,1342.0,Carnegie Mellon University,47,North America,6385.0,13356.0
48,Hong Kong,1027.0,1349.0,City University of Hong Kong,49,Asia,3273.0,9240.0
49,Australia,1829.0,3360.0,The University of Sydney,50,Oceania,17030.0,46678.0


In [30]:
times_ranking_response = cached_get_request("https://www.timeshighereducation.com/sites/default/files/the_data_rankings/world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json")
times_raw_ranking_data = times_ranking_response.json()['data']

def read_times_entry(raw_entry):
    from math import floor
    
    name = raw_entry["name"]
    rank = read_rank(raw_entry["rank"])
    # TODO: Assert this is always a country
    country = raw_entry["location"]
    # No region found
    region = None
    students_total = read_us_formatted_integer(raw_entry["stats_number_students"])
    international_students_ratio = read_percentage(raw_entry["stats_pc_intl_students"])
    students_international = floor(students_total * international_students_ratio)
    faculty_to_students_ratio = read_percentage(raw_entry["stats_student_staff_ratio"])
    faculty_total = floor(students_total * faculty_to_students_ratio)
    faculty_international = None
    
    
    return make_entry(name, rank, country, region, faculty_total, faculty_international, students_total, students_international)
    
#   uniProp['internStudentRatio']=t['stats_pc_intl_students']
#   uniProp['internStudent']=math.floor(float(t['stats_pc_intl_students'].replace("%",""))/100*int(t['stats_number_students'].replace(",","")))#Number looks like this 20,409
#   uniProp['totalFacultyRatio']=t['stats_student_staff_ratio']#TODO define which one we want to use
#   uniProp['totalFaculty']=math.floor(int(t['stats_number_students'].replace(",",""))/float(t['stats_student_staff_ratio']))
#   topBAsObjects.append(uniProp)
    
    result["intern_student"] = None
    
    # TODO define which one we want to use
    result["total_faculty_ratio"] = read_percentage(raw_entry["stats_student_staff_ratio"])
    
    result["total_faculty"] = None
    
    return result

def get_times_data(raw_ranking_data, max_rank):
    """
    Return the list of parsed enries (with details) of the top universities ranking.
    It may perform network requests.
    """
    for raw_entry in raw_ranking_data:
        entry = {}
        
        rank = read_rank(raw_entry["rank"])
        if (rank > max_rank):
            # Assume sorted raw ranks to break (instead of `continue`-ing)
            break

        yield read_times_entry(raw_entry)


def get_times_df(raw_ranking_data, max_rank):
    return pd.DataFrame([*get_times_data(raw_ranking_data, max_rank)])


times_df = get_times_df(times_raw_ranking_data, MAX_RANK)
times_df

,country,faculty_international,faculty_total,name,rank,region,students_international,students_total
0,United Kingdom,None,2285,University of Oxford,1,None,7755,20409
1,United Kingdom,None,2004,University of Cambridge,2,None,6436,18389
2,United States,None,143,California Institute of Technology,3,None,596,2209
3,United States,None,1188,Stanford University,3,None,3485,15845
4,United States,None,972,Massachusetts Institute of Technology,5,None,3800,11177
5,United States,None,1809,Harvard University,6,None,5284,20326
6,United States,None,660,Princeton University,7,None,1909,7955
7,United Kingdom,None,1807,Imperial College London,8,None,8721,15857
8,United States,None,838,University of Chicago,9,None,3381,13525
9,Switzerland,None,2808,ETH Zurich – Swiss Federal Institute of Techno...,10,None,7308,19233


TODO adrien dans le bus, amuse toi bien
Question 3
Commencer par faire un match exact sur le nom et afficher celles qui ne matchent pas -> les garder en exemple et commenter
On ne doit pas aggreger les nombres etc (voir faq http://go.epfl.ch/ada_faq_hw2)
en gros Nom | Classement 1 | Classement 2

Question 4 : 
Calculer tous les pearsons coefficients pour essayer de trouver des corrélations.
A voir si tu trouves/penses à d'autres méthodes
Question 5 : 
Pour le classement global : tester min max mean et regarder les différences, surtout dire pourquoi un est mieux que l'autre
Eventuellement -> il y a pour chaque université des sous classements mais il faut repaser je ferai ça demain

TODO : 
    - Plot question 1 et 2
    - Global ranking différents sous classements 
    - bien commenter
    - analayse textuelle des résultats et de nos idées

# Part 5 
** Que j'auais bien voulu faire mais malheureusement c'est pas merge.......**

First idea : 
Mean of rank numbers.
TODO : Show results and analyse them. Maybe too much equalities or wierd results ?


Second idea : 
Use more precise scores : 
For example we have following scores : 
**Times** 
* overall score
* teaching
* research
* citations
* industry income
* international outlook

**top**
* academic reputation
* citations per faculty
* employer reputation
* faculty student
* international faculty
* international student

We can notice similar categories : 
The most important is overall score. 
We can take just the mean of both of them and make a new ranking.

Scores that are also presents : 
* citations 
    * times : 30% 
    * top : 20%
    * times : We examine research influence by capturing the number of times a university’s published work is cited by scholars globally.
    * Top : To calculate it, we the total number of citations received by all papers produced by an institution across a five-year period by the number of faculty members at that institution.
    * Both are normalized
* international outlook and mean of (international faculty and international student)
    * times : 7,5%
    * Top 2* 5% 
    * Times : calculated like this :
        * International-to-domestic-student ratio: 2.5%
        * International-to-domestic-staff ratio: 2.5%
        * International collaboration: 2.5%
Maybe we can use mean of overall score and take more into account citations and international outlook at it is present in both ranking 


We can also think to electre methods, for example electre III is particulary suited to build a ranking